In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
from matplotlib import cm
from matplotlib import pyplot as plt
from Turbulence_processing.Scaling import *
from pyproj import Proj, Transformer

In [3]:
def select_times(ds):
    hour = ds.time.dt.hour
    night = (hour >= 19) | (hour <= 6)
    day = (hour >= 8) & (hour <= 17)
    return night, day

In [4]:
# Stations dictionary
# -------------------
# open csv
stations = pd.read_csv("Perdigao_station.csv")
tower_names = list(stations.groupby(by="Name").groups.keys())
# export to dictionary
tower_dict = {}
for name in tower_names:
    # single tower dictionary
    station = stations[stations["Name"] == name]
    t_dict = {
        "name": name,
        "lat": station.iloc[0]["Latitude"],
        "lon": station.iloc[0]["Longitude"],
        "east": station.iloc[0]["East"],
        "north": station.iloc[0]["North"],
        "height": station.iloc[0]["Tower_height"],
        "heights_sonic": station[station["Equipment"] == "3D Sonic anemometer"][
            "height"
        ].values,
        "heights_ht": station[
            station["Equipment"] == "Air temperature and humidity sensor"
        ]["height"].values,
        "heights_bar": station[station["Equipment"] == "Barometer"]["height"].values,
    }
    tower_dict[name] = t_dict

In [5]:
# Function to transform coordinates from UTM to a local coordinate system and vice versa
def transform_UTM_to_local(east, north, inverse=False):
    # Create an array of coordinates
    coords = np.array([east, north])

    # Define a rotation matrix for coordinate transformation
    rot = np.array(
        [
            [np.cos(np.pi / 4), np.sin(np.pi / 4)],
            [-np.sin(np.pi / 4), np.cos(np.pi / 4)],
        ]
    )

    # Define the center coordinates for the local coordinate system
    center = np.array([608250, 4396621])

    # two versions for single and array
    if len(np.shape(east)) == 0:
        # Apply the transformation from UTM to local coordinates if inverse is False
        if not inverse:
            coords = rot @ (coords - center).T
        # Apply the inverse transformation from local coordinates to UTM if inverse is True
        else:
            coords = rot.T @ coords.T + center.T
        return coords

    else:
        coords = np.column_stack([east, north])
        new_coords = []
        for i in range(len(coords)):
            # Apply the transformation from UTM to local coordinates if inverse is False
            if not inverse:
                new_coords.append(rot @ (coords[i] - center).T)
            # Apply the inverse transformation from local coordinates to UTM if inverse is True
            else:
                new_coords.append(rot.T @ coords[i].T + center.T)
        return np.array(new_coords)


ETRS_UTM_transform = Transformer.from_crs("EPSG:3763", "EPSG:32629").transform

In [6]:
def get_local_coords(tower):
    # tower coordinates
    E = tower["east"]
    N = tower["north"]
    x_t, y_t = transform_UTM_to_local(*ETRS_UTM_transform(E, N))
    # coordinates offset
    x_t += 239
    y_t += 68
    return (x_t, y_t)

## open data

In [17]:
# function to open and prepare dataset
def open_prepare(tower_name, time_avg):
    # -------------------
    #  OPEN DATA
    # footprint
    ffp_stat = xr.open_dataset(
        f"../../../../../Dataset/Perdigao/Footprint terrain analysis/results/ffp_terrain_{time_avg}_{tower_name}.nc"
    )
    # transect
    trans_stat = xr.open_dataset(
        f"../../../../../Dataset/Perdigao/upwind_transect_analysis/results/trans_terrain_{time_avg}_{tower_name}.nc"
    )
    ds = xr.open_dataset(
        f"../../../../../Dataset/Perdigao/postprocessed_{time_avg}/Perdigao_statistics_{time_avg}_{tower_name}.nc"
    )
    # BLH
    BLH = xr.open_dataset("../../../../../Dataset/Perdigao/BLH_ERA.nc")
    BLH = BLH.interp(time=ds.time.data)
    BLH = xr.DataArray(data=BLH.BLH.data.flatten(), coords={"time": BLH.time})

    # PROCESS
    # ---------------------------------------------------
    # find sonic height
    idx_h = [i for i, x in enumerate(bool_20m_array) if x][0]
    h = heights[idx_h]
    ds = ds.sel(heights=h)
    ffp_stat = ffp_stat.sel(heights=h)
    trans_stat = trans_stat.sel(heights=h)
    
    # assign spatial coordinates
    x_t, y_t = get_local_coords(tower)
    ds = ds.assign_coords({"x": x_t, "y": y_t})

    # select daytime
    ds = ds.where(select_times(ds)[1], drop=True)
    BLH = BLH.where(select_times(BLH)[1], drop=True)
    ffp_stat = ffp_stat.where(select_times(ffp_stat)[1], drop=True)
    trans_stat = trans_stat.where(select_times(trans_stat)[1], drop=True)

    # anisotropy and gradients
    bary, RGB = Anisotropy.Anisotropy(ds, one_sonic=True)
    ds = ds.assign(
        xb=(['time'],bary[0]),
        yb=(['time'],bary[1])
    )

    # ADD VARIABLES
    # --------------------------------------------
    # Obukhov length
    L = -ds.ustar**3 * ds.meanT / (0.4 * 9.81 * ds.wT)
    # free convection velocity
    wfc = (9.81 / ds.meanT * ds.wT * BLH) ** (1 / 3)
    # budget terms
    tkeprod_buoy = 9.81 / ds.meanT * ds.wT

    ds = ds.assign(
        # height / BLH
        z_zi=ds.heights / BLH,
        # zeta stability parameter
        zeta=-1 / L * ds.heights,
        # Saleski roll to cell
        zi_L=BLH / L,
        # zeta with blh (heisel chamecki 23)
        zeta_blh=1 / np.sqrt(-L * BLH) * ds.heights,
        # z / z0
        z_z0=ds.heights / ds.z0,
        # integral time  / memory
        tw_te=ds.intlenW / ds.meanU / ds.tke * ds.epsU,
        tw_te_u_wfc=ds.intlenW / wfc / ds.tke * ds.epsU,
        tw_te_u_ust=ds.intlenW / ds.ustar / ds.tke * ds.epsU,
        # rapid distortion
        rapid_dist_neut=ds.ustar / 0.4 / ds.heights * ds.tke / ds.epsU,
        # free convection velocity
        U_wfc=ds.meanU / wfc,
        # dynamical stuff
        U_ust=ds.meanU / ds.ustar,
        # tke and stress budget
        # production
        tkeprod_buoy_eps=tkeprod_buoy / ds.epsU,
        # lengthscales from Ghannam et al 2018
        ust3_eps_z=ds.ustar**3 / ds.epsU / ds.heights,
        # moments
        kurt_u=ds.uuuu / ds.uu**2,
        kurt_v=ds.vvvv / ds.vv**2,
        kurt_w=ds.wwww / ds.ww**2,
        skew_u=ds.uuu / ds.uu**1.5,
        skew_v=ds.vvv / ds.vv**1.5,
        skew_w=ds.www / ds.ww**1.5,
        uv_tke=ds.uv / ds.tke,
        vw_tke=ds.vw / ds.tke,
        uw_tke=ds.uw / ds.tke,
        uT_wT=ds.uT / ds.wT,
        vT_wT=ds.vT / ds.wT,
        vw_uw=ds.vw / ds.uw,
        uv_uw=ds.uv / ds.uw,
        skew_T=ds.TTT / ds.TT**1.5,
        kurt_T=ds.TTTT / ds.TT**2,
    )
    ds = ds.drop(
        [
            "QCnan",
            "statU",
            "statUW",
            "statWT",
            "meanU",
            "meanT",
            "uu",
            "vv",
            "ww",
            "uv",
            "uw",
            "vw",
            "TT",
            "uT",
            "vT",
            "wT",
            "tke",
            "xb",
            "ustar",
            "theta",
            "phi",
            "uuu",
            "vvv",
            "www",
            "TTT",
            "uuuu",
            "vvvv",
            "wwww",
            "TTTT",
            "epsU",
            "epsV",
            "epsW",
            "epsT",
            "epsUsf",
            "epsVsf",
            "epsWsf",
            "slopeHU",
            "slopeHV",
            "slopeHW",
            "slopeHT",
            "slopeLU",
            "slopeLV",
            "slopeLW",
            "slopeLT",
            "sdir",
            "uuv",
            "uuw",
            "uvw",
            "uvv",
            "uww",
            "vvw",
            "vww",
            "utke",
            "vtke",
            "wtke",
            "uuT",
            "vvT",
            "wwT",
            "uvT",
            "uwT",
            "vwT",
            "uTT",
            "vTT",
            "wTT",
            "dir",
            "intlenW",
            "intlenU",
            "intlenV",
        ]
    )
    # merge and return
    ds = xr.merge([ds, ffp_stat, trans_stat])
    return ds

In [19]:
time_avg = "30min"
datasets = []

for tower_name in tower_names:

    # check if there is sonic at 20 m plus tolerance
    tower = tower_dict[tower_name]
    heights = tower["heights_sonic"]
    bool_20m_array = (heights < 21) & (heights > 19)
    if True in bool_20m_array:
        datasets.append(open_prepare(tower_name, time_avg))


ds = xr.concat(datasets, dim="tower")

In [20]:
ds.to_netcdf("forest_data.nc")

## pass to pandas and select groups
selected for test 8may B1 middle clouds, 23 may S3 clear sky and 11 june W3 clear sky

In [22]:
ds = xr.open_dataset("forest_data.nc")

In [23]:
# stack ds
ds = (
    ds.stack(index=("time", "tower"))
    .reset_index("index")
    .dropna(dim="index", how="any")
)

In [24]:
# hour that starts the day
day_start = 8

# Define group labels
dates = pd.to_datetime(ds.time)
groups = (
    dates
    - pd.to_datetime(datetime(dates[0].year, dates[0].month, dates[0].day, day_start))
).days

ds = ds.assign(groups=(["index"], groups))

# test group
test_groups = [7, 22, 41]

# split
train_bool = np.invert(np.isin(groups, test_groups))
groups_train = groups[train_bool]
ds_train = ds.where(train_bool).dropna(dim="index")
ds_test = ds.where(np.invert(train_bool)).dropna(dim="index")

In [25]:
# Pandas
vars_list = list(ds.data_vars)
vars_list.remove("yb")
X_train = ds_train.to_dataframe()[vars_list]
y_train = ds_train.to_dataframe()["yb"]
X_test = ds_test.to_dataframe()[vars_list]
y_test = ds_test.to_dataframe()["yb"]


# drop groups where not necessary
X_test = X_test.drop(columns="groups")

In [26]:
# save
X_train.to_csv("xtrain.csv")
y_train.to_csv("ytrain.csv")
X_test.to_csv("xtest.csv")
y_test.to_csv("ytest.csv")

In [27]:
# X_train = pd.read_csv("../xtrain.csv", index_col=0)
# y_train = pd.read_csv("../ytrain.csv", index_col=0)
# X_test = pd.read_csv("../xtest.csv", index_col=0)
# y_test = pd.read_csv("../ytest.csv", index_col=0)
# groups = X_train['groups']
# X_train = X_train.drop(columns='groups')